In [1]:
import numpy as np
import gym

In [2]:
env = gym.make("CartPole-v1")

/home/andrea/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
n_x, n_vcart, n_theta, n_vpole = (10,10,10,10)

In [4]:
def map_to_int(val, bounds, n_points):
    if val > bounds[1]:
        return n_points
    if val < bounds[0]:
        return 0
    return round((val - bounds[0])/(bounds[1] - bounds[0])*n_points)

In [10]:
low_bounds, high_bounds = (env.observation_space.low, env.observation_space.high)
bounds_x = (low_bounds[0], high_bounds[0])
bounds_vcart = (0,5)
bounds_theta = (low_bounds[2], high_bounds[2])
bounds_vpole = (0,5)

In [11]:
def Learn(env, brain, stop_rule, epsilon_greedy_rule, print_every_ep=5):

    reward_trajectory = []
    training_step = 0 # Number of updates taht the NN performs
    
    while True: # Cycle over all the episodes until a stop_rule is satisfied
        
        if stop_rule(training_step, reward_trajectory): # Check the training end
            break 
            
        state = env.reset() # Reset environment
        ep_reward = 0 # Reset the score for this episode.
        
        while True: # Cycle over the episode steps

            explore_p = epsilon_greedy_rule(training_step) # Epsilon greedy scheduling
            action = brain.act(state, explore_p) # Get action
            next_state, reward, done, _ = env.step(action) # Take action
            ep_reward += reward # Accumulate reward

            if done: # Episode is completed -- failure or max number of steps reached (success)
                reward_trajectory.append(ep_reward)
                # Add experience to bucket (next_state is None since episode is over).
                brain.train(state, action, None, reward)
                if len(reward_trajectory) % print_every_ep == 0:
                    print("Episode: {}, Steps: {}, Last reward: {}, Explore P: {}".format(len(reward_trajectory), training_step, ep_reward, explore_p))
                break

            brain.train(state, action, next_state, reward) # Store tuple.
            state = next_state # Advance state
            training_step += 1
                
    return reward_trajectory    

In [ ]:
def Q_brain(object):
    
    def __init__(self, state_space_sizes, actions):
        self.s_size = state_space_sizes
        self.a = actions
        self.Q = np.zeros(state_space_sizes)
        
        
    def act()